<a href="https://colab.research.google.com/github/SiMori92/MarketOpen30-DL/blob/main/Step_0_Data_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Content
1. Install Alpha Vantage
2. Testing scripts
3. Executable scripts

### Install Alpha Vantage

In [ ]:
pip install alpha-vantage

### testing scripts

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str, daily_data):
    """Check if the given date is a trading day."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        return date.date() in daily_data.index.date
    except:
        return False

def fetch_alpha_vantage_data(ticker, target_date, api_key):
    """Fetch stock data from Alpha Vantage for a single trading day."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'Pre-market/ Futures Open Price', 'Pre-market/ Futures Price at 08:00',
        'Pre-market/ Futures Volume at 08:00', 'Pre-market/ Futures Close Price',
        'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume',
        'Price at market open',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00',
        'Volume at 9:31', 'Volume at 9:35', 'Volume at 9:40', 'Volume at 9:45',
        'Volume at 9:50', 'Volume at 9:55', 'Volume at 10:00'
    ]
    record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

    # Define timestamps and time ranges outside try block
    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31'), ('0931-0935', '09:31', '09:35'),
        ('0936-0940', '09:36', '09:40'), ('0941-0945', '09:41', '09:45'),
        ('0946-0950', '09:46', '09:50'), ('0951-0955', '09:51', '09:55'),
        ('0956-1000', '09:56', '10:00')
    ]

    # Validate target date
    try:
        target_dt = datetime.strptime(target_date, '%Y-%m-%d')
        if target_dt.date() > datetime.today().date():
            logging.error("Target date is in the future. Use a past or current trading day.")
            return pd.DataFrame([record], columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame([record], columns=columns)

    # Fetch daily data for T-1 fields
    try:
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='compact')
        time.sleep(12)  # Respect rate limit
        daily_data.columns = ['open', 'high', 'low', 'close', 'volume']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date is a trading day
        if not is_trading_day(target_date, daily_data):
            logging.warning(f"{target_date} is not a trading day. Trying previous trading day.")
            target_dt = target_dt - timedelta(days=1)
            target_date = target_dt.strftime('%Y-%m-%d')
            record['Date'] = target_date
            if not is_trading_day(target_date, daily_data):
                logging.error(f"No trading day found for {target_date} or prior.")
                return pd.DataFrame([record], columns=columns)

        # Get previous trading day
        prev_day = target_dt - timedelta(days=1)
        while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
            prev_day -= timedelta(days=1)
        if prev_day.date() in daily_data.index.date:
            prev_day_str = prev_day.strftime('%Y-%m-%d')
            record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close']
            record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high']
            record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low']
            record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume']
        else:
            logging.warning(f"No data for previous trading day before {target_date}")
            record['T-1 Close Price'] = np.nan
            record['T-1 High Price'] = np.nan
            record['T-1 Low Price'] = np.nan
            record['T-1 Volume'] = np.nan
    except Exception as e:
        logging.error(f"Error fetching daily data: {e}")
        record['T-1 Close Price'] = np.nan
        record['T-1 High Price'] = np.nan
        record['T-1 Low Price'] = np.nan
        record['T-1 Volume'] = np.nan

    # Fetch intraday data for the target day
    try:
        intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full')  # Use 'full' for more data
        time.sleep(12)  # Respect rate limit
        intraday.columns = ['open', 'high', 'low', 'close', 'volume']
        intraday.index = pd.to_datetime(intraday.index)
        intraday.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date exists in intraday data
        day_data = intraday[intraday.index.date == target_dt.date()]
        if not day_data.empty:
            # Pre-market data (4:00 AM - 9:29 AM)
            pre_market = day_data.between_time('04:00', '09:29')
            if not pre_market.empty:
                record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                record['Pre-market/ Futures Volume'] = pre_market['volume'].sum()

                # Price and volume at 08:00
                ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                if not pre_market_before_0800.empty:
                    record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                    record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].iloc[-1]
                else:
                    record['Pre-market/ Futures Price at 08:00'] = np.nan
                    record['Pre-market/ Futures Volume at 08:00'] = np.nan
            else:
                logging.warning(f"No pre-market data for {target_date}")
                for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price',
                              'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
                              'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00',
                              'Pre-market/ Futures Volume at 08:00']:
                    record[field] = np.nan

            # Market data (9:30 AM - 10:00 AM)
            market_data = day_data.between_time('09:30', '10:00')
            if not market_data.empty:
                record['Price at market open'] = market_data['open'].iloc[0]

                # Specific timestamps
                for t in timestamps:
                    ts = pd.to_datetime(f"{target_date} {t}:00")
                    market_data_before_t = market_data[market_data.index <= ts]
                    if not market_data_before_t.empty:
                        record[f'Price at {t}'] = market_data_before_t['close'].iloc[-1]
                        record[f'Volume at {t}'] = market_data_before_t['volume'].iloc[-1]
                    else:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan

                # High and low prices for time ranges
                for prefix, start_time, end_time in time_ranges:
                    range_data = market_data.between_time(start_time, end_time)
                    if not range_data.empty:
                        record[f'High Price {prefix}'] = range_data['high'].max()
                        record[f'Low Price {prefix}'] = range_data['low'].min()
                    else:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                record['Price at market open'] = np.nan
                for t in timestamps:
                    record[f'Price at {t}'] = np.nan
                    record[f'Volume at {t}'] = np.nan
                for prefix, _, _ in time_ranges:
                    record[f'High Price {prefix}'] = np.nan
                    record[f'Low Price {prefix}'] = np.nan
        else:
            logging.warning(f"No intraday data for {target_date}. Trying previous trading day.")
            target_dt = target_dt - timedelta(days=1)
            target_date = target_dt.strftime('%Y-%m-%d')
            record['Date'] = target_date
            day_data = intraday[intraday.index.date == target_dt.date()]
            if not day_data.empty:
                # Repeat pre-market and market data processing for new target_date
                pre_market = day_data.between_time('04:00', '09:29')
                if not pre_market.empty:
                    record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                    record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                    record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                    record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                    record['Pre-market/ Futures Volume'] = pre_market['volume'].sum()

                    ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                    pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                    if not pre_market_before_0800.empty:
                        record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                        record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].iloc[-1]
                    else:
                        record['Pre-market/ Futures Price at 08:00'] = np.nan
                        record['Pre-market/ Futures Volume at 08:00'] = np.nan
                else:
                    logging.warning(f"No pre-market data for {target_date}")
                    for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price',
                                  'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
                                  'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00',
                                  'Pre-market/ Futures Volume at 08:00']:
                        record[field] = np.nan

                market_data = day_data.between_time('09:30', '10:00')
                if not market_data.empty:
                    record['Price at market open'] = market_data['open'].iloc[0]
                    for t in timestamps:
                        ts = pd.to_datetime(f"{target_date} {t}:00")
                        market_data_before_t = market_data[market_data.index <= ts]
                        if not market_data_before_t.empty:
                            record[f'Price at {t}'] = market_data_before_t['close'].iloc[-1]
                            record[f'Volume at {t}'] = market_data_before_t['volume'].iloc[-1]
                        else:
                            record[f'Price at {t}'] = np.nan
                            record[f'Volume at {t}'] = np.nan
                    for prefix, start_time, end_time in time_ranges:
                        range_data = market_data.between_time(start_time, end_time)
                        if not range_data.empty:
                            record[f'High Price {prefix}'] = range_data['high'].max()
                            record[f'Low Price {prefix}'] = range_data['low'].min()
                        else:
                            record[f'High Price {prefix}'] = np.nan
                            record[f'Low Price {prefix}'] = np.nan
                else:
                    logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                    record['Price at market open'] = np.nan
                    for t in timestamps:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan
                    for prefix, _, _ in time_ranges:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No intraday data for {target_date}")
                for field in columns[7:]:
                    record[field] = np.nan
    except Exception as e:
        logging.error(f"Error fetching intraday data: {e}")
        for field in columns[7:]:
            record[field] = np.nan

    return pd.DataFrame([record], columns=columns)

def main():
    # Configurable parameters
    ticker = 'SOXL'  # Hardcoded for debugging
    api_key = 'DP6XIR4OLYVAZVBC'  # Replace with your valid API key
    target_date = '2025-06-17'  # Use a recent trading day for testing

    # Fetch and save data
    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, target_date, api_key)
    df.to_csv(f'{ticker}_finance_data_{target_date}.csv', index=False)
    logging.info(f"Data saved to {ticker}_finance_data_{target_date}.csv")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str, daily_data):
    """Check if the given date is a trading day."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        return date.date() in daily_data.index.date
    except:
        return False

def fetch_alpha_vantage_data(ticker, target_date, api_key):
    """Fetch stock data from Alpha Vantage for a single trading day or return nulls if not a trading day."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'Pre-market/ Futures Open Price', 'Pre-market/ Futures Price at 08:00',
        'Pre-market/ Futures Volume at 08:00', 'Pre-market/ Futures Close Price',
        'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume',
        'Price at market open',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00',
        'Volume at 9:31', 'Volume at 9:35', 'Volume at 9:40', 'Volume at 9:45',
        'Volume at 9:50', 'Volume at 9:55', 'Volume at 10:00'
    ]
    record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

    # Define timestamps and time ranges
    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31'), ('0931-0935', '09:31', '09:35'),
        ('0936-0940', '09:36', '09:40'), ('0941-0945', '09:41', '09:45'),
        ('0946-0950', '09:46', '09:50'), ('0951-0955', '09:51', '09:55'),
        ('0956-1000', '09:56', '10:00')
    ]

    # Validate target date
    try:
        target_dt = datetime.strptime(target_date, '%Y-%m-%d')
        if target_dt.date() > datetime.today().date():
            logging.error("Target date is in the future. Use a past or current trading day.")
            return pd.DataFrame([record], columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame([record], columns=columns)

    # Fetch daily data for T-1 fields and to check trading day
    try:
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='compact')
        time.sleep(12)  # Respect rate limit
        daily_data.columns = ['open', 'high', 'low', 'close', 'volume']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date is a trading day
        if not is_trading_day(target_date, daily_data):
            logging.warning(f"{target_date} is not a trading day. Returning null values.")
            for field in columns[3:]:  # Skip ID, Date, Ticker
                record[field] = np.nan
            return pd.DataFrame([record], columns=columns)

        # Get previous trading day
        prev_day = target_dt - timedelta(days=1)
        while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
            prev_day -= timedelta(days=1)
        if prev_day.date() in daily_data.index.date:
            prev_day_str = prev_day.strftime('%Y-%m-%d')
            record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close']
            record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high']
            record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low']
            record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume']
        else:
            logging.warning(f"No data for previous trading day before {target_date}")
            record['T-1 Close Price'] = np.nan
            record['T-1 High Price'] = np.nan
            record['T-1 Low Price'] = np.nan
            record['T-1 Volume'] = np.nan
    except Exception as e:
        logging.error(f"Error fetching daily data: {e}")
        record['T-1 Close Price'] = np.nan
        record['T-1 High Price'] = np.nan
        record['T-1 Low Price'] = np.nan
        record['T-1 Volume'] = np.nan

    # Fetch intraday data for the target day
    try:
        intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full')
        time.sleep(12)  # Respect rate limit
        intraday.columns = ['open', 'high', 'low', 'close', 'volume']
        intraday.index = pd.to_datetime(intraday.index)
        intraday.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date exists in intraday data
        day_data = intraday[intraday.index.date == target_dt.date()]
        if not day_data.empty:
            # Pre-market data (4:00 AM - 9:29 AM)
            pre_market = day_data.between_time('04:00', '09:29')
            if not pre_market.empty:
                record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                record['Pre-market/ Futures Volume'] = pre_market['volume'].sum()

                # Price and volume at 08:00
                ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                if not pre_market_before_0800.empty:
                    record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                    record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].iloc[-1]
                else:
                    record['Pre-market/ Futures Price at 08:00'] = np.nan
                    record['Pre-market/ Futures Volume at 08:00'] = np.nan
            else:
                logging.warning(f"No pre-market data for {target_date}")
                for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price',
                              'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
                              'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00',
                              'Pre-market/ Futures Volume at 08:00']:
                    record[field] = np.nan

            # Market data (9:30 AM - 10:00 AM)
            market_data = day_data.between_time('09:30', '10:00')
            if not market_data.empty:
                record['Price at market open'] = market_data['open'].iloc[0]

               # Specific timestamps
                for t in timestamps:
                    ts = pd.to_datetime(f"{target_date} {t}:00")
                    market_data_before_t = market_data[market_data.index <= ts]
                    """ if not market_data_before_t.empty:
                        record[f'Price at {t}'] = market_data_before_t['close'].iloc[-1]
                        record[f'Volume at {t}'] = market_data_before_t['volume'].iloc[-1]
                    else:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan """

                # High and low prices for time ranges
                for prefix, start_time, end_time in time_ranges:
                    range_data = market_data.between_time(start_time, end_time)
                    if not range_data.empty:
                        record[f'High Price {prefix}'] = range_data['high'].max()
                        record[f'Low Price {prefix}'] = range_data['low'].min()
                        record[f'Close Price {prefix}'] = range_data['close'].iloc[-1]
                        record[f'Volume {prefix}'] = range_data['volume'].iloc[0]
                    else:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
                        record[f'Close Price {prefix}'] = np.nan
                        record[f'Volume {prefix}'] = np.nan
            else:
                logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                record['Price at market open'] = np.nan
                for t in timestamps:
                    record[f'Price at {t}'] = np.nan
                    record[f'Volume at {t}'] = np.nan
                for prefix, _, _ in time_ranges:
                    record[f'High Price {prefix}'] = np.nan
                    record[f'Low Price {prefix}'] = np.nan
        else:
            logging.warning(f"No intraday data for {target_date}")
            for field in columns[7:]:
                record[field] = np.nan
    except Exception as e:
        logging.error(f"Error fetching intraday data: {e}")
        for field in columns[7:]:
            record[field] = np.nan

    return pd.DataFrame([record], columns=columns)

def main():
    # Configurable parameters
    ticker = 'SOXL'  # Hardcoded for debugging
    api_key = 'DP6XIR4OLYVAZVBC'  # Replace with your valid API key
    target_date = '2025-06-13'  # Use a recent trading day for testing

    # Fetch and save data
    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, target_date, api_key)
    df.to_csv(f'{ticker}_finance_data_{target_date}.csv', index=False)
    logging.info(f"Data saved to {ticker}_finance_data_{target_date}.csv")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str, daily_data):
    """Check if the given date is a trading day."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        return date.date() in daily_data.index.date
    except:
        return False

def fetch_alpha_vantage_data(ticker, target_date, api_key):
    """Fetch stock data from Alpha Vantage for a single trading day or return nulls if not a trading day."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'Pre-market/ Futures Open Price', 'Pre-market/ Futures Price at 08:00',
        'Pre-market/ Futures Volume at 08:00', 'Pre-market/ Futures Close Price',
        'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume',
        'Price at market open',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

    # Define timestamps and time ranges
    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    # Validate target date
    try:
        target_dt = datetime.strptime(target_date, '%Y-%m-%d')
        if target_dt.date() > datetime.today().date():
            logging.error("Target date is in the future. Use a past or current trading day.")
            return pd.DataFrame([record], columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame([record], columns=columns)

    # Fetch daily data for T-1 fields and to check trading day
    try:
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='compact')
        time.sleep(12)  # Respect rate limit
        daily_data.columns = ['open', 'high', 'low', 'close', 'volume']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date is a trading day
        if not is_trading_day(target_date, daily_data):
            logging.warning(f"{target_date} is not a trading day. Returning null values.")
            for field in columns[3:]:  # Skip ID, Date, Ticker
                record[field] = np.nan
            return pd.DataFrame([record], columns=columns)

        # Get previous trading day
        prev_day = target_dt - timedelta(days=1)
        while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
            prev_day -= timedelta(days=1)
        if prev_day.date() in daily_data.index.date:
            prev_day_str = prev_day.strftime('%Y-%m-%d')
            record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close']
            record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high']
            record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low']
            record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume']
        else:
            logging.warning(f"No data for previous trading day before {target_date}")
            record['T-1 Close Price'] = np.nan
            record['T-1 High Price'] = np.nan
            record['T-1 Low Price'] = np.nan
            record['T-1 Volume'] = np.nan
    except Exception as e:
        logging.error(f"Error fetching daily data: {e}")
        record['T-1 Close Price'] = np.nan
        record['T-1 High Price'] = np.nan
        record['T-1 Low Price'] = np.nan
        record['T-1 Volume'] = np.nan

    # Fetch intraday data for the target day
    try:
        intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full')
        time.sleep(12)  # Respect rate limit
        intraday.columns = ['open', 'high', 'low', 'close', 'volume']
        intraday.index = pd.to_datetime(intraday.index)
        intraday.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date exists in intraday data
        day_data = intraday[intraday.index.date == target_dt.date()]
        if not day_data.empty:
            # Pre-market data (4:00 AM - 9:29 AM)
            pre_market = day_data.between_time('04:00', '09:29')
            if not pre_market.empty:
                record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                record['Pre-market/ Futures Volume'] = pre_market['volume'].sum()

                # Price and volume at 08:00
                ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                if not pre_market_before_0800.empty:
                    record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                    record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum()
                else:
                    record['Pre-market/ Futures Price at 08:00'] = np.nan
                    record['Pre-market/ Futures Volume at 08:00'] = np.nan
            else:
                logging.warning(f"No pre-market data for {target_date}")
                for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price',
                              'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
                              'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00',
                              'Pre-market/ Futures Volume at 08:00']:
                    record[field] = np.nan

            # Market data (9:30 AM - 10:00 AM)
            market_data = day_data.between_time('09:30', '10:00')
            if not market_data.empty:
                record['Price at market open'] = market_data['open'].iloc[0]
                logging.info(f"Available market data timestamps: {market_data.index}")

                # Process time ranges for high, low, close, and volume
                for prefix, start_time, end_time, timestamp in time_ranges:
                    range_data = market_data.between_time(start_time, end_time)
                    if not range_data.empty:
                        record[f'High Price {prefix}'] = range_data['high'].max()
                        record[f'Low Price {prefix}'] = range_data['low'].min()
                        record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                        record[f'Volume at {timestamp}'] = range_data['volume'].sum()
                        logging.info(f"Time range {prefix} ({start_time}-{end_time}): "
                                     f"Price={record[f'Price at {timestamp}']}, "
                                     f"Volume={record[f'Volume at {timestamp}']}, "
                                     f"Rows={len(range_data)}")
                    else:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
                        record[f'Price at {timestamp}'] = np.nan
                        record[f'Volume at {timestamp}'] = np.nan
                        logging.warning(f"No data for time range {prefix} ({start_time}-{end_time})")
            else:
                logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                record['Price at market open'] = np.nan
                for t in timestamps:
                    record[f'Price at {t}'] = np.nan
                    record[f'Volume at {t}'] = np.nan
                for prefix, _, _, _ in time_ranges:
                    record[f'High Price {prefix}'] = np.nan
                    record[f'Low Price {prefix}'] = np.nan
        else:
            logging.warning(f"No intraday data for {target_date}")
            for field in columns[7:]:
                record[field] = np.nan
    except Exception as e:
        logging.error(f"Error fetching intraday data: {e}")
        for field in columns[7:]:
            record[field] = np.nan

    return pd.DataFrame([record], columns=columns)

def main():
    # Configurable parameters
    ticker = 'SOXL'  # Hardcoded for debugging
    api_key = 'DP6XIR4OLYVAZVBC'  # Replace with your valid API key
    target_date = '2017-01-11'  # Use a recent trading day for testing

    # Fetch and save data
    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, target_date, api_key)
    df.to_csv(f'{ticker}_finance_data_{target_date}.csv', index=False)
    logging.info(f"Data saved to {ticker}_finance_data_{target_date}.csv")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str):
    """Check if the given date is likely a trading day based on calendar."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        # Check for weekends (Saturday=5, Sunday=6)
        if date.weekday() >= 5:
            return False
        # Hardcoded major US holidays for 2017
        holidays_2017 = [
            '2017-01-02',  # New Year's Day (observed)
            '2017-01-16',  # Martin Luther King Jr. Day
            '2017-02-20',  # Presidents' Day
            '2017-04-14',  # Good Friday
            '2017-05-29',  # Memorial Day
            '2017-07-04',  # Independence Day
            '2017-09-04',  # Labor Day
            '2017-11-23',  # Thanksgiving
            '2017-12-25'   # Christmas
        ]
        return date_str not in holidays_2017
    except ValueError:
        return False

def fetch_alpha_vantage_data(ticker, target_date, api_key):
    """Fetch stock data from Alpha Vantage for a single trading day."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'Pre-market/ Futures Open Price', 'Pre-market/ Futures Price at 08:00',
        'Pre-market/ Futures Volume at 08:00', 'Pre-market/ Futures Close Price',
        'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume',
        'Price at market open',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

    # Define timestamps and time ranges
    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    # Validate target date
    try:
        target_dt = datetime.strptime(target_date, '%Y-%m-%d')
        if target_dt.date() > datetime.today().date():
            logging.error("Target date is in the future. Use a past or current trading day.")
            return pd.DataFrame([record], columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame([record], columns=columns)

    # Check if target date is a trading day
    if not is_trading_day(target_date):
        logging.warning(f"{target_date} is not a trading day (weekend or holiday). Returning null values.")
        for field in columns[3:]:  # Skip ID, Date, Ticker
            record[field] = np.nan
        return pd.DataFrame([record], columns=columns)

    # Fetch daily data for T-1 fields and to check data availability
    try:
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
        time.sleep(12)  # Respect rate limit
        daily_data.columns = ['open', 'high', 'low', 'close', 'volume']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date has data
        if target_dt.date() not in daily_data.index.date:
            logging.warning(f"{target_date} is a trading day, but no daily data available from Alpha Vantage. Returning null values.")
            for field in columns[3:]:  # Skip ID, Date, Ticker
                record[field] = np.nan
            return pd.DataFrame([record], columns=columns)

        # Get previous trading day
        prev_day = target_dt - timedelta(days=1)
        while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
            prev_day -= timedelta(days=1)
        if prev_day.date() in daily_data.index.date:
            prev_day_str = prev_day.strftime('%Y-%m-%d')
            record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close']
            record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high']
            record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low']
            record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume']
        else:
            logging.warning(f"No data for previous trading day before {target_date}")
            record['T-1 Close Price'] = np.nan
            record['T-1 High Price'] = np.nan
            record['T-1 Low Price'] = np.nan
            record['T-1 Volume'] = np.nan

        # Set market open price for target date
        target_day_str = target_dt.strftime('%Y-%m-%d')
        if target_day_str in daily_data.index:
            record['Price at market open'] = daily_data.loc[target_day_str, 'open']
        else:
            record['Price at market open'] = np.nan
    except Exception as e:
        logging.error(f"Error fetching daily data: {e}")
        record['T-1 Close Price'] = np.nan
        record['T-1 High Price'] = np.nan
        record['T-1 Low Price'] = np.nan
        record['T-1 Volume'] = np.nan
        record['Price at market open'] = np.nan

    # Fetch intraday data for the target day with month parameter
    try:
        month_param = target_dt.strftime('%Y-%m')  # e.g., '2017-01'
        intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full', month=month_param)
        time.sleep(12)  # Respect rate limit
        intraday.columns = ['open', 'high', 'low', 'close', 'volume']
        intraday.index = pd.to_datetime(intraday.index)
        intraday.sort_index(inplace=True)  # Ensure index is monotonic

        # Check if target date exists in intraday data
        day_data = intraday[intraday.index.date == target_dt.date()]
        if not day_data.empty:
            # Pre-market data (4:00 AM - 9:29 AM)
            pre_market = day_data.between_time('04:00', '09:29')
            if not pre_market.empty:
                record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                record['Pre-market/ Futures Volume'] = pre_market['volume'].sum()

                # Price and volume at 08:00
                ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                if not pre_market_before_0800.empty:
                    record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                    record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum()
                else:
                    record['Pre-market/ Futures Price at 08:00'] = np.nan
                    record['Pre-market/ Futures Volume at 08:00'] = np.nan
            else:
                logging.warning(f"No pre-market data for {target_date}")
                for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price',
                              'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
                              'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00',
                              'Pre-market/ Futures Volume at 08:00']:
                    record[field] = np.nan

            # Market data (9:30 AM - 10:00 AM)
            market_data = day_data.between_time('09:30', '10:00')
            if not market_data.empty:
                logging.info(f"Available market data timestamps: {market_data.index}")

                # Process time ranges for high, low, close, and volume
                for prefix, start_time, end_time, timestamp in time_ranges:
                    range_data = market_data.between_time(start_time, end_time)
                    if not range_data.empty:
                        record[f'High Price {prefix}'] = range_data['high'].max()
                        record[f'Low Price {prefix}'] = range_data['low'].min()
                        record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                        record[f'Volume at {timestamp}'] = range_data['volume'].sum()
                        logging.info(f"Time range {prefix} ({start_time}-{end_time}): "
                                     f"Price={record[f'Price at {timestamp}']}, "
                                     f"Volume={record[f'Volume at {timestamp}']}, "
                                     f"Rows={len(range_data)}")
                    else:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
                        record[f'Price at {timestamp}'] = np.nan
                        record[f'Volume at {timestamp}'] = np.nan
                        logging.warning(f"No data for time range {prefix} ({start_time}-{end_time})")
            else:
                logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                for t in timestamps:
                    record[f'Price at {t}'] = np.nan
                    record[f'Volume at {t}'] = np.nan
                for prefix, _, _, _ in time_ranges:
                    record[f'High Price {prefix}'] = np.nan
                    record[f'Low Price {prefix}'] = np.nan
        else:
            logging.warning(f"No intraday data for {target_date} in month {month_param}")
            for field in columns[7:]:
                record[field] = np.nan
    except Exception as e:
        logging.error(f"Error fetching intraday data: {e}")
        for field in columns[7:]:
            record[field] = np.nan

    return pd.DataFrame([record], columns=columns)

def main():
    # Configurable parameters
    ticker = 'SOXL'
    api_key = 'DP6XIR4OLYVAZVBC'  # Replace with your valid API key
    target_date = '2017-01-04'

    # Fetch and save data
    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, target_date, api_key)
    df.to_csv(f'{ticker}_finance_data_{target_date}.csv', index=False)
    logging.info(f"Data saved to {ticker}_finance_data_{target_date}.csv")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

### Final script - after premium API

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str):
    """Check if the given date is likely a trading day based on calendar."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        if date.weekday() >= 5:
            return False
        holidays = [
            '-01-01', '-01-16', '-02-20', '-04-14', '-05-29', '-07-04', '-09-04', '-11-23', '-12-25'
        ]
        holidays_full = []
        for year in range(2017, 2026):
            holidays_full.extend([f"{year}{h}" for h in holidays])
        holidays_full.extend(['2017-01-02', '2023-01-02'])
        return date_str not in holidays_full
    except ValueError:
        return False

def fetch_alpha_vantage_data(ticker, start_date, end_date, api_key):
    """Fetch stock data from Alpha Vantage for a date range, skipping non-trading days, with all prices and volumes adjusted for splits and dividends."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'T-1 Adjusted Close Price', 'T-1 Adjusted Volume', 'Pre-market/ Futures Open Price',
        'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00',
        'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
        'Pre-market/ Futures Volume', 'Price at market open', 'Adjusted Close Price',
        'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    records = []

    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        if start_dt > end_dt:
            logging.error("Start date must be before end date.")
            return pd.DataFrame(columns=columns)
        if end_dt.date() > datetime.today().date():
            logging.error("End date is in the future. Use a past or current date.")
            return pd.DataFrame(columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame(columns=columns)

    try:
        daily_data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        time.sleep(0.8)
        daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)
    except Exception as e:
        logging.error(f"Error fetching daily adjusted data: {e}")
        daily_data = pd.DataFrame()

    current_dt = start_dt
    while current_dt <= end_dt:
        target_date = current_dt.strftime('%Y-%m-%d')
        record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

        if not is_trading_day(target_date):
            logging.info(f"Skipping {target_date}: not a trading day (weekend or holiday).")
            current_dt += timedelta(days=1)
            continue

        if not daily_data.empty and current_dt.date() not in daily_data.index.date:
            logging.warning(f"{target_date} is a trading day, but no daily data available from Alpha Vantage.")
            for field in columns[3:]:
                record[field] = np.nan
            records.append(record)
            current_dt += timedelta(days=1)
            continue

        target_day_str = current_dt.strftime('%Y-%m-%d')
        split_coefficient = daily_data.loc[target_day_str, 'split_coefficient'] if target_day_str in daily_data.index else 1.0
        adjustment_factor = (daily_data.loc[target_day_str, 'close'] / daily_data.loc[target_day_str, 'adjusted_close']) if target_day_str in daily_data.index and daily_data.loc[target_day_str, 'adjusted_close'] != 0 else 1.0

        try:
            prev_day = current_dt - timedelta(days=1)
            while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
                prev_day -= timedelta(days=1)
            if prev_day.date() in daily_data.index.date:
                prev_day_str = prev_day.strftime('%Y-%m-%d')
                prev_adjustment_factor = (daily_data.loc[prev_day_str, 'close'] / daily_data.loc[prev_day_str, 'adjusted_close']) if daily_data.loc[prev_day_str, 'adjusted_close'] != 0 else 1.0
                prev_split_coefficient = daily_data.loc[prev_day_str, 'split_coefficient']
                record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close'] / prev_adjustment_factor
                record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high'] / prev_adjustment_factor
                record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low'] / prev_adjustment_factor
                record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
                record['T-1 Adjusted Close Price'] = daily_data.loc[prev_day_str, 'adjusted_close']
                record['T-1 Adjusted Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
            else:
                logging.warning(f"No data for previous trading day before {target_date}")
                for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error processing T-1 data for {target_date}: {e}")
            for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                record[field] = np.nan

        try:
            if target_day_str in daily_data.index:
                record['Price at market open'] = daily_data.loc[target_day_str, 'open'] / adjustment_factor
                record['Adjusted Close Price'] = daily_data.loc[target_day_str, 'close'] / adjustment_factor
                record['Split/Dividend Adjusted Close Price'] = daily_data.loc[target_day_str, 'adjusted_close']
                record['Split/Dividend Adjusted Volume'] = daily_data.loc[target_day_str, 'volume'] / split_coefficient
            else:
                for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching daily data for {target_date}: {e}")
            for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                record[field] = np.nan

        try:
            month_param = current_dt.strftime('%Y-%m')
            intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full', month=month_param)
            time.sleep(0.8)
            intraday.columns = ['open', 'high', 'low', 'close', 'volume']
            intraday.index = pd.to_datetime(intraday.index)
            intraday.sort_index(inplace=True)

            day_data = intraday[intraday.index.date == current_dt.date()]
            if not day_data.empty:
                pre_market = day_data.between_time('04:00', '09:29')
                if not pre_market.empty:
                    record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                    record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                    record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                    record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                    record['Pre-market/ Futures Volume'] = pre_market['volume'].sum() / split_coefficient
                    ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                    pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                    if not pre_market_before_0800.empty:
                        record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                        record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum() / split_coefficient
                    else:
                        record['Pre-market/ Futures Price at 08:00'] = np.nan
                        record['Pre-market/ Futures Volume at 08:00'] = np.nan
                else:
                    logging.warning(f"No pre-market data for {target_date}")
                    for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00']:
                        record[field] = np.nan

                market_data = day_data.between_time('09:30', '10:00')
                if not market_data.empty:
                    logging.info(f"Available market data timestamps for {target_date}: {market_data.index}")
                    for prefix, start_time, end_time, timestamp in time_ranges:
                        range_data = market_data.between_time(start_time, end_time)
                        if not range_data.empty:
                            record[f'High Price {prefix}'] = range_data['high'].max()
                            record[f'Low Price {prefix}'] = range_data['low'].min()
                            record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                            record[f'Volume at {timestamp}'] = range_data['volume'].sum() / split_coefficient
                            logging.info(f"Time range {prefix} ({start_time}-{end_time}) for {target_date}: "
                                         f"Price={record[f'Price at {timestamp}']}, "
                                         f"Volume={record[f'Volume at {timestamp}']}, "
                                         f"Rows={len(range_data)}")
                        else:
                            record[f'High Price {prefix}'] = np.nan
                            record[f'Low Price {prefix}'] = np.nan
                            record[f'Price at {timestamp}'] = np.nan
                            record[f'Volume at {timestamp}'] = np.nan
                            logging.warning(f"No data for time range {prefix} ({start_time}-{end_time}) for {target_date}")
                else:
                    logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                    for t in timestamps:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan
                    for prefix, _, _, _ in time_ranges:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No intraday data for {target_date} in month {month_param}")
                for field in columns[9:]:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching intraday data for {target_date}: {e}")
            for field in columns[9:]:
                record[field] = np.nan

        records.append(record)
        current_dt += timedelta(days=1)

    return pd.DataFrame(records, columns=columns)

def main():
    ticker = 'SOXL'
    api_key = 'X9WCBUVBIX5S6GX7'
    start_date = '2017-01-01'
    end_date = '2025-06-20'

    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, start_date, end_date, api_key)
    output_file = f'{ticker}_finance_data_{start_date}_to_{end_date}.csv'
    df.to_csv(output_file, index=False)
    logging.info(f"Data saved to {output_file}")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

ERROR:root:Error fetching intraday data for 2021-10-04: Error getting data from the api, no return was given.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str):
    """Check if the given date is likely a trading day based on calendar."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        if date.weekday() >= 5:
            return False
        holidays = [
            '-01-01', '-01-16', '-02-20', '-04-14', '-05-29', '-07-04', '-09-04', '-11-23', '-12-25'
        ]
        holidays_full = []
        for year in range(2017, 2026):
            holidays_full.extend([f"{year}{h}" for h in holidays])
        holidays_full.extend(['2017-01-02', '2023-01-02'])
        return date_str not in holidays_full
    except ValueError:
        return False

def fetch_alpha_vantage_data(ticker, start_date, end_date, api_key):
    """Fetch stock data from Alpha Vantage for a date range, skipping non-trading days, with all prices and volumes adjusted for splits and dividends."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'T-1 Adjusted Close Price', 'T-1 Adjusted Volume', 'Pre-market/ Futures Open Price',
        'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00',
        'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
        'Pre-market/ Futures Volume', 'Price at market open', 'Adjusted Close Price',
        'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    records = []

    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        if start_dt > end_dt:
            logging.error("Start date must be before end date.")
            return pd.DataFrame(columns=columns)
        if end_dt.date() > datetime.today().date():
            logging.error("End date is in the future. Use a past or current date.")
            return pd.DataFrame(columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame(columns=columns)

    try:
        daily_data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        time.sleep(0.8)
        daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)
    except Exception as e:
        logging.error(f"Error fetching daily adjusted data: {e}")
        daily_data = pd.DataFrame()

    current_dt = start_dt
    while current_dt <= end_dt:
        target_date = current_dt.strftime('%Y-%m-%d')
        record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

        if not is_trading_day(target_date):
            logging.info(f"Skipping {target_date}: not a trading day (weekend or holiday).")
            current_dt += timedelta(days=1)
            continue

        if not daily_data.empty and current_dt.date() not in daily_data.index.date:
            logging.warning(f"{target_date} is a trading day, but no daily data available from Alpha Vantage.")
            for field in columns[3:]:
                record[field] = np.nan
            records.append(record)
            current_dt += timedelta(days=1)
            continue

        target_day_str = current_dt.strftime('%Y-%m-%d')
        split_coefficient = daily_data.loc[target_day_str, 'split_coefficient'] if target_day_str in daily_data.index else 1.0
        adjustment_factor = (daily_data.loc[target_day_str, 'close'] / daily_data.loc[target_day_str, 'adjusted_close']) if target_day_str in daily_data.index and daily_data.loc[target_day_str, 'adjusted_close'] != 0 else 1.0

        try:
            prev_day = current_dt - timedelta(days=1)
            while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
                prev_day -= timedelta(days=1)
            if prev_day.date() in daily_data.index.date:
                prev_day_str = prev_day.strftime('%Y-%m-%d')
                prev_adjustment_factor = (daily_data.loc[prev_day_str, 'close'] / daily_data.loc[prev_day_str, 'adjusted_close']) if daily_data.loc[prev_day_str, 'adjusted_close'] != 0 else 1.0
                prev_split_coefficient = daily_data.loc[prev_day_str, 'split_coefficient']
                record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close'] / prev_adjustment_factor
                record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high'] / prev_adjustment_factor
                record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low'] / prev_adjustment_factor
                record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
                record['T-1 Adjusted Close Price'] = daily_data.loc[prev_day_str, 'adjusted_close']
                record['T-1 Adjusted Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
            else:
                logging.warning(f"No data for previous trading day before {target_date}")
                for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error processing T-1 data for {target_date}: {e}")
            for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                record[field] = np.nan

        try:
            if target_day_str in daily_data.index:
                record['Price at market open'] = daily_data.loc[target_day_str, 'open'] / adjustment_factor
                record['Adjusted Close Price'] = daily_data.loc[target_day_str, 'close'] / adjustment_factor
                record['Split/Dividend Adjusted Close Price'] = daily_data.loc[target_day_str, 'adjusted_close']
                record['Split/Dividend Adjusted Volume'] = daily_data.loc[target_day_str, 'volume'] / split_coefficient
            else:
                for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching daily data for {target_date}: {e}")
            for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                record[field] = np.nan

        try:
            month_param = current_dt.strftime('%Y-%m')
            intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full', month=month_param)
            time.sleep(0.8)
            intraday.columns = ['open', 'high', 'low', 'close', 'volume']
            intraday.index = pd.to_datetime(intraday.index)
            intraday.sort_index(inplace=True)

            day_data = intraday[intraday.index.date == current_dt.date()]
            if not day_data.empty:
                pre_market = day_data.between_time('04:00', '09:29')
                if not pre_market.empty:
                    record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                    record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                    record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                    record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                    record['Pre-market/ Futures Volume'] = pre_market['volume'].sum() / split_coefficient
                    ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                    pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                    if not pre_market_before_0800.empty:
                        record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                        record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum() / split_coefficient
                    else:
                        record['Pre-market/ Futures Price at 08:00'] = np.nan
                        record['Pre-market/ Futures Volume at 08:00'] = np.nan
                else:
                    logging.warning(f"No pre-market data for {target_date}")
                    for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00']:
                        record[field] = np.nan

                market_data = day_data.between_time('09:30', '10:00')
                if not market_data.empty:
                    logging.info(f"Available market data timestamps for {target_date}: {market_data.index}")
                    for prefix, start_time, end_time, timestamp in time_ranges:
                        range_data = market_data.between_time(start_time, end_time)
                        if not range_data.empty:
                            record[f'High Price {prefix}'] = range_data['high'].max()
                            record[f'Low Price {prefix}'] = range_data['low'].min()
                            record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                            record[f'Volume at {timestamp}'] = range_data['volume'].sum() / split_coefficient
                            logging.info(f"Time range {prefix} ({start_time}-{end_time}) for {target_date}: "
                                         f"Price={record[f'Price at {timestamp}']}, "
                                         f"Volume={record[f'Volume at {timestamp}']}, "
                                         f"Rows={len(range_data)}")
                        else:
                            record[f'High Price {prefix}'] = np.nan
                            record[f'Low Price {prefix}'] = np.nan
                            record[f'Price at {timestamp}'] = np.nan
                            record[f'Volume at {timestamp}'] = np.nan
                            logging.warning(f"No data for time range {prefix} ({start_time}-{end_time}) for {target_date}")
                else:
                    logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                    for t in timestamps:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan
                    for prefix, _, _, _ in time_ranges:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No intraday data for {target_date} in month {month_param}")
                for field in columns[9:]:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching intraday data for {target_date}: {e}")
            for field in columns[9:]:
                record[field] = np.nan

        records.append(record)
        current_dt += timedelta(days=1)

    return pd.DataFrame(records, columns=columns)

def main():
    ticker = 'TQQQ'
    api_key = 'X9WCBUVBIX5S6GX7'
    start_date = '2017-01-01'
    end_date = '2025-06-20'

    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, start_date, end_date, api_key)
    output_file = f'{ticker}_finance_data_{start_date}_to_{end_date}.csv'
    df.to_csv(output_file, index=False)
    logging.info(f"Data saved to {output_file}")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

ERROR:root:Error fetching intraday data for 2021-03-26: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.
ERROR:root:Error fetching intraday data for 2023-05-26: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.
ERROR:root:Error fetching intraday data for 2025-04-30: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str):
    """Check if the given date is likely a trading day based on calendar."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        if date.weekday() >= 5:
            return False
        holidays = [
            '-01-01', '-01-16', '-02-20', '-04-14', '-05-29', '-07-04', '-09-04', '-11-23', '-12-25'
        ]
        holidays_full = []
        for year in range(2017, 2026):
            holidays_full.extend([f"{year}{h}" for h in holidays])
        holidays_full.extend(['2017-01-02', '2023-01-02'])
        return date_str not in holidays_full
    except ValueError:
        return False

def fetch_alpha_vantage_data(ticker, start_date, end_date, api_key):
    """Fetch stock data from Alpha Vantage for a date range, skipping non-trading days, with all prices and volumes adjusted for splits and dividends."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'T-1 Adjusted Close Price', 'T-1 Adjusted Volume', 'Pre-market/ Futures Open Price',
        'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00',
        'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
        'Pre-market/ Futures Volume', 'Price at market open', 'Adjusted Close Price',
        'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    records = []

    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        if start_dt > end_dt:
            logging.error("Start date must be before end date.")
            return pd.DataFrame(columns=columns)
        if end_dt.date() > datetime.today().date():
            logging.error("End date is in the future. Use a past or current date.")
            return pd.DataFrame(columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame(columns=columns)

    try:
        daily_data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        time.sleep(0.8)
        daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)
    except Exception as e:
        logging.error(f"Error fetching daily adjusted data: {e}")
        daily_data = pd.DataFrame()

    current_dt = start_dt
    while current_dt <= end_dt:
        target_date = current_dt.strftime('%Y-%m-%d')
        record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

        if not is_trading_day(target_date):
            logging.info(f"Skipping {target_date}: not a trading day (weekend or holiday).")
            current_dt += timedelta(days=1)
            continue

        if not daily_data.empty and current_dt.date() not in daily_data.index.date:
            logging.warning(f"{target_date} is a trading day, but no daily data available from Alpha Vantage.")
            for field in columns[3:]:
                record[field] = np.nan
            records.append(record)
            current_dt += timedelta(days=1)
            continue

        target_day_str = current_dt.strftime('%Y-%m-%d')
        split_coefficient = daily_data.loc[target_day_str, 'split_coefficient'] if target_day_str in daily_data.index else 1.0
        adjustment_factor = (daily_data.loc[target_day_str, 'close'] / daily_data.loc[target_day_str, 'adjusted_close']) if target_day_str in daily_data.index and daily_data.loc[target_day_str, 'adjusted_close'] != 0 else 1.0

        try:
            prev_day = current_dt - timedelta(days=1)
            while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
                prev_day -= timedelta(days=1)
            if prev_day.date() in daily_data.index.date:
                prev_day_str = prev_day.strftime('%Y-%m-%d')
                prev_adjustment_factor = (daily_data.loc[prev_day_str, 'close'] / daily_data.loc[prev_day_str, 'adjusted_close']) if daily_data.loc[prev_day_str, 'adjusted_close'] != 0 else 1.0
                prev_split_coefficient = daily_data.loc[prev_day_str, 'split_coefficient']
                record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close'] / prev_adjustment_factor
                record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high'] / prev_adjustment_factor
                record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low'] / prev_adjustment_factor
                record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
                record['T-1 Adjusted Close Price'] = daily_data.loc[prev_day_str, 'adjusted_close']
                record['T-1 Adjusted Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
            else:
                logging.warning(f"No data for previous trading day before {target_date}")
                for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error processing T-1 data for {target_date}: {e}")
            for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                record[field] = np.nan

        try:
            if target_day_str in daily_data.index:
                record['Price at market open'] = daily_data.loc[target_day_str, 'open'] / adjustment_factor
                record['Adjusted Close Price'] = daily_data.loc[target_day_str, 'close'] / adjustment_factor
                record['Split/Dividend Adjusted Close Price'] = daily_data.loc[target_day_str, 'adjusted_close']
                record['Split/Dividend Adjusted Volume'] = daily_data.loc[target_day_str, 'volume'] / split_coefficient
            else:
                for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching daily data for {target_date}: {e}")
            for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                record[field] = np.nan

        try:
            month_param = current_dt.strftime('%Y-%m')
            intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full', month=month_param)
            time.sleep(0.8)
            intraday.columns = ['open', 'high', 'low', 'close', 'volume']
            intraday.index = pd.to_datetime(intraday.index)
            intraday.sort_index(inplace=True)

            day_data = intraday[intraday.index.date == current_dt.date()]
            if not day_data.empty:
                pre_market = day_data.between_time('04:00', '09:29')
                if not pre_market.empty:
                    record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                    record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                    record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                    record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                    record['Pre-market/ Futures Volume'] = pre_market['volume'].sum() / split_coefficient
                    ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                    pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                    if not pre_market_before_0800.empty:
                        record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                        record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum() / split_coefficient
                    else:
                        record['Pre-market/ Futures Price at 08:00'] = np.nan
                        record['Pre-market/ Futures Volume at 08:00'] = np.nan
                else:
                    logging.warning(f"No pre-market data for {target_date}")
                    for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00']:
                        record[field] = np.nan

                market_data = day_data.between_time('09:30', '10:00')
                if not market_data.empty:
                    logging.info(f"Available market data timestamps for {target_date}: {market_data.index}")
                    for prefix, start_time, end_time, timestamp in time_ranges:
                        range_data = market_data.between_time(start_time, end_time)
                        if not range_data.empty:
                            record[f'High Price {prefix}'] = range_data['high'].max()
                            record[f'Low Price {prefix}'] = range_data['low'].min()
                            record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                            record[f'Volume at {timestamp}'] = range_data['volume'].sum() / split_coefficient
                            logging.info(f"Time range {prefix} ({start_time}-{end_time}) for {target_date}: "
                                         f"Price={record[f'Price at {timestamp}']}, "
                                         f"Volume={record[f'Volume at {timestamp}']}, "
                                         f"Rows={len(range_data)}")
                        else:
                            record[f'High Price {prefix}'] = np.nan
                            record[f'Low Price {prefix}'] = np.nan
                            record[f'Price at {timestamp}'] = np.nan
                            record[f'Volume at {timestamp}'] = np.nan
                            logging.warning(f"No data for time range {prefix} ({start_time}-{end_time}) for {target_date}")
                else:
                    logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                    for t in timestamps:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan
                    for prefix, _, _, _ in time_ranges:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No intraday data for {target_date} in month {month_param}")
                for field in columns[9:]:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching intraday data for {target_date}: {e}")
            for field in columns[9:]:
                record[field] = np.nan

        records.append(record)
        current_dt += timedelta(days=1)

    return pd.DataFrame(records, columns=columns)

def main():
    ticker = 'UVXY'
    api_key = 'X9WCBUVBIX5S6GX7'
    start_date = '2017-01-01'
    end_date = '2025-06-20'

    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, start_date, end_date, api_key)
    output_file = f'{ticker}_finance_data_{start_date}_to_{end_date}.csv'
    df.to_csv(output_file, index=False)
    logging.info(f"Data saved to {output_file}")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid
from alpha_vantage.timeseries import TimeSeries
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def is_trading_day(date_str):
    """Check if the given date is likely a trading day based on calendar."""
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        if date.weekday() >= 5:
            return False
        holidays = [
            '-01-01', '-01-16', '-02-20', '-04-14', '-05-29', '-07-04', '-09-04', '-11-23', '-12-25'
        ]
        holidays_full = []
        for year in range(2017, 2026):
            holidays_full.extend([f"{year}{h}" for h in holidays])
        holidays_full.extend(['2017-01-02', '2023-01-02'])
        return date_str not in holidays_full
    except ValueError:
        return False

def fetch_alpha_vantage_data(ticker, start_date, end_date, api_key):
    """Fetch stock data from Alpha Vantage for a date range, skipping non-trading days, with all prices and volumes adjusted for splits and dividends."""
    ts = TimeSeries(key=api_key, output_format='pandas')
    columns = [
        'ID', 'Date', 'Ticker', 'T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume',
        'T-1 Adjusted Close Price', 'T-1 Adjusted Volume', 'Pre-market/ Futures Open Price',
        'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00',
        'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price',
        'Pre-market/ Futures Volume', 'Price at market open', 'Adjusted Close Price',
        'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume',
        'High Price 0930-0931', 'Low Price 0930-0931', 'Price at 9:31', 'Volume at 9:31',
        'High Price 0931-0935', 'Low Price 0931-0935', 'Price at 9:35', 'Volume at 9:35',
        'High Price 0936-0940', 'Low Price 0936-0940', 'Price at 9:40', 'Volume at 9:40',
        'High Price 0941-0945', 'Low Price 0941-0945', 'Price at 9:45', 'Volume at 9:45',
        'High Price 0946-0950', 'Low Price 0946-0950', 'Price at 9:50', 'Volume at 9:50',
        'High Price 0951-0955', 'Low Price 0951-0955', 'Price at 9:55', 'Volume at 9:55',
        'High Price 0956-1000', 'Low Price 0956-1000', 'Price at 10:00', 'Volume at 10:00'
    ]
    records = []

    timestamps = ['09:31', '09:35', '09:40', '09:45', '09:50', '09:55', '10:00']
    time_ranges = [
        ('0930-0931', '09:30', '09:31', '9:31'),
        ('0931-0935', '09:31', '09:35', '9:35'),
        ('0936-0940', '09:36', '09:40', '9:40'),
        ('0941-0945', '09:41', '09:45', '9:45'),
        ('0946-0950', '09:46', '09:50', '9:50'),
        ('0951-0955', '09:51', '09:55', '9:55'),
        ('0956-1000', '09:56', '10:00', '10:00')
    ]

    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        if start_dt > end_dt:
            logging.error("Start date must be before end date.")
            return pd.DataFrame(columns=columns)
        if end_dt.date() > datetime.today().date():
            logging.error("End date is in the future. Use a past or current date.")
            return pd.DataFrame(columns=columns)
    except ValueError:
        logging.error("Invalid date format. Use YYYY-MM-DD.")
        return pd.DataFrame(columns=columns)

    try:
        daily_data, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        time.sleep(0.8)
        daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
        daily_data.index = pd.to_datetime(daily_data.index)
        daily_data.sort_index(inplace=True)
    except Exception as e:
        logging.error(f"Error fetching daily adjusted data: {e}")
        daily_data = pd.DataFrame()

    current_dt = start_dt
    while current_dt <= end_dt:
        target_date = current_dt.strftime('%Y-%m-%d')
        record = {'ID': str(uuid.uuid4()), 'Date': target_date, 'Ticker': ticker}

        if not is_trading_day(target_date):
            logging.info(f"Skipping {target_date}: not a trading day (weekend or holiday).")
            current_dt += timedelta(days=1)
            continue

        if not daily_data.empty and current_dt.date() not in daily_data.index.date:
            logging.warning(f"{target_date} is a trading day, but no daily data available from Alpha Vantage.")
            for field in columns[3:]:
                record[field] = np.nan
            records.append(record)
            current_dt += timedelta(days=1)
            continue

        target_day_str = current_dt.strftime('%Y-%m-%d')
        split_coefficient = daily_data.loc[target_day_str, 'split_coefficient'] if target_day_str in daily_data.index else 1.0
        adjustment_factor = (daily_data.loc[target_day_str, 'close'] / daily_data.loc[target_day_str, 'adjusted_close']) if target_day_str in daily_data.index and daily_data.loc[target_day_str, 'adjusted_close'] != 0 else 1.0

        try:
            prev_day = current_dt - timedelta(days=1)
            while prev_day.date() not in daily_data.index.date and prev_day >= daily_data.index.min():
                prev_day -= timedelta(days=1)
            if prev_day.date() in daily_data.index.date:
                prev_day_str = prev_day.strftime('%Y-%m-%d')
                prev_adjustment_factor = (daily_data.loc[prev_day_str, 'close'] / daily_data.loc[prev_day_str, 'adjusted_close']) if daily_data.loc[prev_day_str, 'adjusted_close'] != 0 else 1.0
                prev_split_coefficient = daily_data.loc[prev_day_str, 'split_coefficient']
                record['T-1 Close Price'] = daily_data.loc[prev_day_str, 'close'] / prev_adjustment_factor
                record['T-1 High Price'] = daily_data.loc[prev_day_str, 'high'] / prev_adjustment_factor
                record['T-1 Low Price'] = daily_data.loc[prev_day_str, 'low'] / prev_adjustment_factor
                record['T-1 Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
                record['T-1 Adjusted Close Price'] = daily_data.loc[prev_day_str, 'adjusted_close']
                record['T-1 Adjusted Volume'] = daily_data.loc[prev_day_str, 'volume'] / prev_split_coefficient
            else:
                logging.warning(f"No data for previous trading day before {target_date}")
                for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error processing T-1 data for {target_date}: {e}")
            for field in ['T-1 Close Price', 'T-1 High Price', 'T-1 Low Price', 'T-1 Volume', 'T-1 Adjusted Close Price', 'T-1 Adjusted Volume']:
                record[field] = np.nan

        try:
            if target_day_str in daily_data.index:
                record['Price at market open'] = daily_data.loc[target_day_str, 'open'] / adjustment_factor
                record['Adjusted Close Price'] = daily_data.loc[target_day_str, 'close'] / adjustment_factor
                record['Split/Dividend Adjusted Close Price'] = daily_data.loc[target_day_str, 'adjusted_close']
                record['Split/Dividend Adjusted Volume'] = daily_data.loc[target_day_str, 'volume'] / split_coefficient
            else:
                for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching daily data for {target_date}: {e}")
            for field in ['Price at market open', 'Adjusted Close Price', 'Split/Dividend Adjusted Close Price', 'Split/Dividend Adjusted Volume']:
                record[field] = np.nan

        try:
            month_param = current_dt.strftime('%Y-%m')
            intraday, _ = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full', month=month_param)
            time.sleep(0.8)
            intraday.columns = ['open', 'high', 'low', 'close', 'volume']
            intraday.index = pd.to_datetime(intraday.index)
            intraday.sort_index(inplace=True)

            day_data = intraday[intraday.index.date == current_dt.date()]
            if not day_data.empty:
                pre_market = day_data.between_time('04:00', '09:29')
                if not pre_market.empty:
                    record['Pre-market/ Futures Open Price'] = pre_market['open'].iloc[0]
                    record['Pre-market/ Futures Close Price'] = pre_market['close'].iloc[-1]
                    record['Pre-market/ Futures High Price'] = pre_market['high'].max()
                    record['Pre-market/ Futures Low Price'] = pre_market['low'].min()
                    record['Pre-market/ Futures Volume'] = pre_market['volume'].sum() / split_coefficient
                    ts_0800 = pd.to_datetime(f"{target_date} 08:00:00")
                    pre_market_before_0800 = pre_market[pre_market.index <= ts_0800]
                    if not pre_market_before_0800.empty:
                        record['Pre-market/ Futures Price at 08:00'] = pre_market_before_0800['close'].iloc[-1]
                        record['Pre-market/ Futures Volume at 08:00'] = pre_market_before_0800['volume'].sum() / split_coefficient
                    else:
                        record['Pre-market/ Futures Price at 08:00'] = np.nan
                        record['Pre-market/ Futures Volume at 08:00'] = np.nan
                else:
                    logging.warning(f"No pre-market data for {target_date}")
                    for field in ['Pre-market/ Futures Open Price', 'Pre-market/ Futures Close Price', 'Pre-market/ Futures High Price', 'Pre-market/ Futures Low Price', 'Pre-market/ Futures Volume', 'Pre-market/ Futures Price at 08:00', 'Pre-market/ Futures Volume at 08:00']:
                        record[field] = np.nan

                market_data = day_data.between_time('09:30', '10:00')
                if not market_data.empty:
                    logging.info(f"Available market data timestamps for {target_date}: {market_data.index}")
                    for prefix, start_time, end_time, timestamp in time_ranges:
                        range_data = market_data.between_time(start_time, end_time)
                        if not range_data.empty:
                            record[f'High Price {prefix}'] = range_data['high'].max()
                            record[f'Low Price {prefix}'] = range_data['low'].min()
                            record[f'Price at {timestamp}'] = range_data['close'].iloc[-1]
                            record[f'Volume at {timestamp}'] = range_data['volume'].sum() / split_coefficient
                            logging.info(f"Time range {prefix} ({start_time}-{end_time}) for {target_date}: "
                                         f"Price={record[f'Price at {timestamp}']}, "
                                         f"Volume={record[f'Volume at {timestamp}']}, "
                                         f"Rows={len(range_data)}")
                        else:
                            record[f'High Price {prefix}'] = np.nan
                            record[f'Low Price {prefix}'] = np.nan
                            record[f'Price at {timestamp}'] = np.nan
                            record[f'Volume at {timestamp}'] = np.nan
                            logging.warning(f"No data for time range {prefix} ({start_time}-{end_time}) for {target_date}")
                else:
                    logging.warning(f"No market data for {target_date} between 9:30 and 10:00")
                    for t in timestamps:
                        record[f'Price at {t}'] = np.nan
                        record[f'Volume at {t}'] = np.nan
                    for prefix, _, _, _ in time_ranges:
                        record[f'High Price {prefix}'] = np.nan
                        record[f'Low Price {prefix}'] = np.nan
            else:
                logging.warning(f"No intraday data for {target_date} in month {month_param}")
                for field in columns[9:]:
                    record[field] = np.nan
        except Exception as e:
            logging.error(f"Error fetching intraday data for {target_date}: {e}")
            for field in columns[9:]:
                record[field] = np.nan

        records.append(record)
        current_dt += timedelta(days=1)

    return pd.DataFrame(records, columns=columns)

def main():
    ticker = 'VXX'
    api_key = 'X9WCBUVBIX5S6GX7'
    start_date = '2017-01-01'
    end_date = '2025-06-20'

    start_time = time.time()
    df = fetch_alpha_vantage_data(ticker, start_date, end_date, api_key)
    output_file = f'{ticker}_finance_data_{start_date}_to_{end_date}.csv'
    df.to_csv(output_file, index=False)
    logging.info(f"Data saved to {output_file}")
    logging.info(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def fetch_etf_data(ticker, start_date, end_date):
    """
    Fetch ETF data using yfinance for a given ticker and date range.

    Args:
        ticker (str): ETF ticker symbol
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        DataFrame: Contains Date, Ticker, Open Price, Close Price, High Price, Low Price, Volume
    """
    # Download data using yfinance
    etf = yf.Ticker(ticker)
    data = etf.history(start=start_date, end=end_date)

    # Reset index to make Date a column
    data = data.reset_index()

    # Select and rename columns to match requirements
    data = data[['Date', 'Open', 'Close', 'High', 'Low', 'Volume']]
    data['Ticker'] = ticker
    data = data[['Date', 'Ticker', 'Open', 'Close', 'High', 'Low', 'Volume']]

    # Ensure correct data types
    data['Date'] = pd.to_datetime(data['Date']).dt.date
    data['Ticker'] = data['Ticker'].astype(str)
    data['Open'] = data['Open'].astype(float)
    data['Close'] = data['Close'].astype(float)
    data['High'] = data['High'].astype(float)
    data['Low'] = data['Low'].astype(float)
    data['Volume'] = data['Volume'].astype(float)

    # Rename columns to match variable names
    data.columns = ['Date', 'Ticker', 'Open Price', 'Close Price', 'High Price', 'Low Price', 'Volume']

    return data

# Example usage
if __name__ == "__main__":
    # Define parameters
    ticker = "^VIX"  # Example ETF ticker (SPDR S&P 500 ETF Trust)
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = "2017-01-01"

    # Fetch data
    etf_data = fetch_etf_data(ticker, start_date, end_date)

    # Display data
    print(f"ETF Data for {ticker}:")
    print(etf_data)

    # Optionally save to CSV
    etf_data.to_csv(f"{ticker}_data.csv", index=False)
    print(f"Data saved to {ticker}_data.csv")

ETF Data for VIX:
           Date Ticker   Open Price  Close Price    High Price    Low Price  \
0    2017-01-03    VIX  4952.379883  5238.100098   5238.100098  4761.899902   
1    2017-01-04    VIX  5142.859863  5238.100098   5333.330078  5142.859863   
2    2017-01-05    VIX  5142.859863  5238.100098   5238.100098  5047.620117   
3    2017-01-06    VIX  5142.859863  5047.620117   5238.100098  5047.620117   
4    2017-01-09    VIX  4952.379883  5047.620117   5142.859863  4952.379883   
..          ...    ...          ...          ...           ...          ...   
261  2018-01-22    VIX  9900.000000  9800.000000  10000.000000  9600.000000   
262  2018-01-23    VIX  9900.000000  9900.000000   9900.000000  9700.000000   
263  2018-01-26    VIX  9300.000000  9300.000000   9500.000000  8400.000000   
264  2018-01-30    VIX  9700.000000  9300.000000   9700.000000  9300.000000   
265  2018-01-31    VIX  9400.000000  9300.000000   9400.000000  9100.000000   

       Volume  
0    217416.0  
1